# Descargas

In [ ]:
!git clone https://github.com/Tio-Panda/INF471-Feria.git

In [ ]:
!pip install --no-input git+https://github.com/facebookresearch/sam2.git@2b90b9f5ceec907a1c18123530e92e794ad901a4

# Init

In [ ]:
import sys

EXTENSIONS = [".jpg", ".jpeg", ".png"]
base_path = "/content/INF471-Feria"
sys.path.append(f"{base_path}/code")

In [ ]:
import copy
import random
import numpy as np
from numpy.random import default_rng

from img_handlers import plot_imgs_tuple_array, get_imgs_PIL_array, DensityPalette, PrefabricLoader, get_image_from_solution
from generative import eval_function

# Cargar imágenes

In [ ]:
b_single_imgs_path = f"{base_path}/imgs/b_imgs/textures"
b_full_imgs_path = f"{base_path}/imgs/b_imgs/full_textures"

density_palette = DensityPalette(b_single_imgs_path, b_full_imgs_path, 5, 5, 7)
density_palette.show_palette()

In [ ]:
a_imgs_path = f"{base_path}/imgs/a_imgs"

a_imgs_array = get_imgs_PIL_array(a_imgs_path)
plot_imgs_tuple_array(list(enumerate(a_imgs_array)), "index: {}", height=3, width=3)

# Obtener mapa de figuras, canvas principal y paleta de densidades

In [ ]:
from img_handlers import MainCanvas

idx = 0
canvas = MainCanvas(a_imgs_array[idx], 854, 480)

In [ ]:
from engine import Engine

engine = Engine(
    engine="SAM",
    points_per_side=24,
    points_per_batch=24,
    pred_iou_thresh=0.78,
    stability_score_thresh=0.83,
    stability_score_offset=0.83,
    crop_n_layers=1,
    box_nms_thresh=0.81,
    crop_n_points_downscale_factor=1,
    min_mask_region_area=10,
    use_m2m=True,
)

In [ ]:
shape_map = engine.get_shape_map(canvas.get_canvas())

In [ ]:
selected_range = (1, -1)
shape_map.set_map_sub_polygons(n_max=50, l_high=8000, l_low=500, range=selected_range)
plot_imgs_tuple_array(list(enumerate([
    canvas.get_canvas(), 
    shape_map.get_map_img((0, -1)), 
    shape_map.get_map_img(selected_range),
    shape_map.get_sub_polygons_img(selected_range)
])), "", height=5, width=8)

# Guardar canvas y mapa de figuras

In [ ]:
pf = PrefabricLoader(f"{base_path}/prefabrics")

name = "nombre"
pf.save_prefabric(canvas, shape_map, name)

# Algoritmos Generativos

In [ ]:
canvas_img = canvas.get_canvas()
N, M = canvas_img.size

all_polygons = shape_map.get_concatenated_sub_polygons()
n_polygons = len(all_polygons)

polygons_patches = []
for poly in all_polygons:
    if random.random() < 0.96:
        polygons_patches.append(poly.get_img_with_texture(canvas, density_palette))
    else:
        polygons_patches.append(canvas.get_masked_img(poly.mask))

solution_img = get_image_from_solution(polygons_patches, canvas)

In [ ]:
def generic_algorithm(canvas, all_polygons, polygons_patches_original, density_palette, n_iterations=100, n_mutate_max=1, alpha=0.45, beta=4, w=0.56):
    
    historial = []
    n_polygons = len(all_polygons)
    _polygons_patches = copy.deepcopy(polygons_patches_original)
    
    for _ in range(n_iterations):
        
        change_idx_list = random.choices(range(n_polygons), k=n_mutate_max)

        for idx in change_idx_list:
            solution_img = get_image_from_solution(_polygons_patches, canvas)
            historial.append(solution_img)
            global_eval = eval_function(canvas_img, solution_img, w)
            patch_eval = eval_function(canvas.get_masked_img(all_polygons[idx].mask), _polygons_patches[idx], w)

            deficit = max(0.0, (global_eval - patch_eval)) / (global_eval + 1e-9)
            p = 1 / (1 + np.exp(-beta*(deficit - 0.5)))

            if random.random() >= p:
                if random.random() < alpha:
                    _polygons_patches[idx] = all_polygons[idx].get_img_with_texture(canvas, density_palette)
                else:
                    _polygons_patches[idx] = canvas.get_masked_img(all_polygons[idx].mask)

    return (_polygons_patches, historial)

(_polygons_patches, historial) = generic_algorithm(canvas, all_polygons, polygons_patches, density_palette)

solution_img = get_image_from_solution(_polygons_patches, canvas)

In [ ]:
n_historial = len(historial)
h_idxs = np.linspace(0, n_historial - 1, 16, dtype=int)
show_historial = [historial[i] for i in h_idxs]

plot_imgs_tuple_array(list(zip(h_idxs, show_historial)), title_format="idx: {}", height=3, width=3)

In [ ]:
from img_handlers import get_img_with_bg, row_fusion, col_fusion, diagonal_fusion

original_img = canvas.get_canvas()
w, h = original_img.size

out1 = get_img_with_bg(solution_img, canvas.bg_letter, (255, 255, 255, 255))
out1 = canvas.colorize_img(out1, 0.35)

out2 = get_img_with_bg(solution_img, canvas.bg_grid, (255, 255, 255, 255))
out2 = canvas.colorize_img(out2, 0.15)

out3 = row_fusion(out1, out2, 50, 0.6)

out4 = col_fusion(out1, out2, 100, 0.5)
out4 = row_fusion(out4, canvas.get_canvas(), 50, 0.7)

out5 = diagonal_fusion(out1, out2, 100, 30, 0.5)

out6 = diagonal_fusion(out1, original_img, 90, 40, 0.4)
out6 = diagonal_fusion(out6, out2, 90, -40, 0.6)

plot_imgs_tuple_array(list(enumerate([ 
    solution_img,
    canvas.bg_letter,
    canvas.bg_grid,
    out1,
    out2,
    out3,
    out4,
    out5,
    out6
])), "", height=5, width=8)